In [1]:
#AlphaGo

In [2]:
#Basic Go Game

In [4]:
## The number of spots per side of the board
## This code allows for an nxn board
boardsize = 5
## Value determining whether the player wants to quit or not
gameon = 1
## Lists of groups that have been removed from the board via capture,
## held in these varaibles in case, when all captures have been
## completed, the board resembles a previous game state and
## the move is invalid.  In that case, the groups are restored
## from these varaibles.
restore_o = []
restore_x = []
 
## Generates blank game states
def initalize():
    gs = []
    for i in range(0,boardsize):
        gs.append([])
        for j in range(0,boardsize):
            gs[i].append('-')
    return gs
 
## Provides an ascii display of the Go board
def printboard(gs):
    global boardsize
    for row in gs:
        rowprint = ''
        for element in row:
            rowprint += element
            rowprint += ' '
        print(rowprint)
 
## Returns a list of the board positions surrounding the
## passed group.
def gperm(group):
    permimeter = []
    global boardsize
    hit = 0
    loss = 0
    ## Adds permimeter spots below
    ## Works by looking from top to bottom, left to right,
    ## at each posisition on the board.  When a posistion
    ## is hit that is in the given group, I set hit = 1.
    ## Then, at the next position that is not in that group,
    ## or if the end of the column is reached, I set loss = 1.
    ## That point is the first point below a point in that group,
    ## so it is part of the permieter of that group.
    i = 0
    j = 0
    while i < boardsize:
        j = 0
        hit = 0
        while j < boardsize:
            if [i,j] in group:
                hit = 1
            elif (hit == 1) & ([i,j] not in group):
                loss = 1
            if (hit == 1) & (loss == 1):
                permimeter.append([i,j])
                hit = 0
                loss = 0
            j += 1
        i += 1
    ## Adds permimeter spots to the right
    i = 0
    j = 0
    while i < boardsize:
        j = 0
        hit = 0
        while j < boardsize:
            if [j,i] in group:
                hit = 1
            elif (hit == 1) & ([j,i] not in group):
                loss = 1
            if (hit == 1) & (loss == 1):
                permimeter.append([j,i])
                hit = 0
                loss = 0
            j += 1
        i += 1
    ## Adds permimeter spots above
    i = 0
    j = boardsize-1
    while i < boardsize:
        j = boardsize-1
        hit = 0
        while j >= 0:
            if [i,j] in group:
                hit = 1
            elif (hit == 1) & ([i,j] not in group):
                loss = 1
            if (hit == 1) & (loss == 1):
                permimeter.append([i,j])
                hit = 0
                loss = 0
            j -= 1
        i += 1
    ## Adds permimeter spots to the left
    i = 0
    j = boardsize-1
    while i < boardsize:
        j = boardsize-1
        hit = 0
        while j >= 0:
            if [j,i] in group:
                hit = 1
            elif (hit == 1) & ([j,i] not in group):
                loss = 1
            if (hit == 1) & (loss == 1):
                permimeter.append([j,i])
                hit = 0
                loss = 0
            j -= 1
        i += 1
    return permimeter
 
## Returns a string that describes the game state
def readable(gs):
    readthis = ''
    readthis += '<<'
    for row in gs:
        for element in row:
            readthis += element
    readthis += '>>'
    return readthis
 
## Counts the territory captured by each player
def count():
    global gsc
    global non_groups
    global o_points
    global x_points
    global boardsize
 
    ## Creates a list of groups (non_groups) of empty positions.
    for i in range(0,boardsize):
        for j in range(0,boardsize):
            if gsc[j][i] == '-':
                new = 1
                for group in non_groups:
                    if [i,j] in gperm(group):
                        group.append([i,j])
                        new = 0
                if new == 1:
                    non_groups.append([[i,j]])
    concat('-')
 
    o_points = 0
    x_points = 0
 
    ## Gives a point to the each player for every pebble they have
    ## on the board.
    for group in o_groups:
        o_points += len(group)
    for group in x_groups:
        x_points += len(group)
 
    ## The permimeter of these empty positions is here considered,
    ## and if every position in the permimeter of a non_group is
    ## one player or the other, that player gains a number of points
    ## equal to the length of that group (the number of positions
    ## that their pieces enclose).
    for group in non_groups:
        no = 0
        for element in gperm(group):
            if gsc[element[1]][element[0]] != 'o':
                no = 1
        if no == 0:
            o_points += len(group)
 
    for group in non_groups:
        no = 0
        for element in gperm(group):
            if gsc[element[1]][element[0]] != 'x':
                no = 1
        if no == 0:
            x_points += len(group)
 
## Checks for capture, and removes the captured pieces from the board
def capture(xoro):
    global o_groups
    global x_groups
    global gsf
    global restore_o
    global restore_x
    global edited
    if xoro == 'o':
        groups = x_groups
        otherplayer = 'o'
    else:
        groups = o_groups
        otherplayer = 'x'
 
    ## Checks to see, for each group of a particular player,
    ## whether any of the board positions in the
    ## perimeter around that group are held by the other player.
    ## If any position is not held by the other player,
    ## the group is not captured, and is safe.  Otherwise,
    ## the group is removed.  But we haven't tested this yet
    ## to see if this would return the board to a previous
    ## state, so we save the removed groups with the restore lists.
    for group in groups:
        safe = 0
        for element in gperm(group):
            if gsf[element[1]][element[0]] != otherplayer:
                safe = 1
        if safe != 1:
            edited = 1
            if xoro == 'o':
                restore_x.append(group)
            else:
                restore_o.append(group)
            groups.remove(group)
 
    # Sets gsf given the new captures
    gsf = initalize()
    for group in o_groups:
        for point in group:
            gsf[point[1]][point[0]] = 'o'
    for group in x_groups:
        for point in group:
            gsf[point[1]][point[0]] = 'x'
 
## Checks to see if the new game state, created by the most recent
## move, returns the board to a previous state.  If not, then
## gsc is set as this new state, and gsp is set as what gsc was, and
## the new game state is stored in gscache.  The function returns 1
## if the move is valid, 0 otherwise.
def goodmove():
    global gscache
    global gsc
    global gsp
    global gsf
    if readable(gsf) not in gscache:
        gsp = []
        gsc = []
        for element in gsf:
            gsp.append(element)
            gsc.append(element)
        gscache += readable(gsf)
        return 1
    else:
        return 0
 
## Checks if any groups contain the same point;
## if so, joins them into one group
def concat(xoro):
    global o_groups
    global x_groups
    global non_groups
    if xoro == 'o':
        groups = o_groups
    elif xoro == 'x':
        groups = x_groups
    else:
        groups = non_groups
    i = 0
    ## currentgroups and previousgroups are used to compare the number
    ## of groups before this nest of whiles to the number after.  If
    ## The number is the same, then nothing needed to be concatinated,
    ## and we can move on.  If the number is different, two groups
    ## were concatinated, and we need to run through this nest again
    ## to see if any other groups need to be joined together.
    currentgroups = len(groups)
    previousgroups = currentgroups + 1
    ## Checks if the positions contained in any group are to be
    ## found in any other group.  If so, all elements of the second are
    ## added to the first, and the first is deleted.
    while previousgroups != currentgroups:
        while i < len(groups)-1:
            reset = 0
            j = i + 1
            while j < len(groups):
                k = 0
                while k < len(groups[i]):
                    if groups[i][k] in groups[j]:
                        for element in groups[j]:
                            if element not in groups[i]:
                                groups[i].append(element)
                        groups.remove(groups[j])
                        reset = 1
                    if reset == 1:
                        break
                    k += 1
                j += 1
            if reset == 1:
                i = -1
            i += 1
        previousgroups = currentgroups
        currentgroups = len(groups)
 
## Adds point xy to a group if xy is in the
## perimeter of an existing group, or creates
## new group if xy is not a part of any existing group.
def addpoint(xy,xoro):
    global o_groups
    global x_groups
    if xoro == 'o':
        groups = o_groups
    else:
        groups = x_groups
    new = 1
    for group in groups:
        if xy in gperm(group):
            group.append(xy)
            new = 0
    if new == 1:
        groups.append([xy])
 
## Lets the player select a move.
def selectmove(xoro):
    global boardsize
    global gsf
    hold = 1
    while hold == 1:
 
        minihold = 1
        while minihold == 1:
            pp = raw_input('Place or pass (l/a)? ')
            if pp == 'a':
                return 'pass'
            elif pp == 'l':
                minihold = 0
                ## This try...except ensures that the user
                ## raw_inputs only numbers
                error = 0
                try:
                    x = int(raw_input('x: '))
                except ValueError:
                    error = 1
                try:
                    y = int(raw_input('y: '))
                except ValueError:
                    error = 1
                if error == 1:
                    minihold = 1
                    print('invalid')
            else:
                print('invalid')
        ## Ensures that the raw_input is on the board
        if (x > boardsize) | (x < 0) | (y > boardsize) | (y < 0):
            print('invalid')
        elif gsc[y][x] != '-':
            print('invalid')
        else:
            hold = 0
    ## Places the piece on the 'future' board, the board
    ## used to test if a move is valid
    if xoro == 'o':
        gsf[y][x] = 'o'
    else:
        gsf[y][x] = 'x'
 
    return [x,y]
 
## The 'turn,' in which a player makes a move,
## the captures caused by that piece are made,
## the validity of the move is checked, and
## the endgame status is checked.
def turn():
    global xoro
    global notxoro
    global player1_pass
    global player2_pass
    global gameover
    hold = 1
    while hold == 1:
        print()
        print('place for '+xoro)
        ## By calling selectmove(), the player
        ## is given the option of whether to place
        ## a piece or to pass, and where to place
        ## that piece.
        xy = selectmove(xoro)
        if xy == 'pass':
            if xoro == 'o':
                player1_pass = 1
            else:
                player2_pass = 1
            hold = 0
        ## If the player doesn't pass...
        else:
            player1_pass = 0
            player2_pass = 0
            ## The new piece is added to its group,
            ## or a new group is created for it.
            addpoint(xy,xoro)
            ## Groups that have been connected by
            ## the this placement are joined together
            concat(xoro)
            minihold = 1
            ## Edited is a value used to check
            ## whether any capture is made.  capture()
            ## is called as many times as until no pieces
            ## are capture (until edited does not change
            ## to 1)
            edited = 0
            while minihold == 1:
                restore_o = []
                restore_x = []
                capture(xoro)
                capture(notxoro)
                if edited == 0:
                    minihold = 0
                    edited = 0
                else:
                    edited = 0
            ## Checks to see if the move, given all the
            ## captures it causes, would return the board
            ## to a previous game state.
            if goodmove() == 1:
                hold = 0
            ## If the move is invalid, the captured groups need
            ## to be returned to the board, so we use
            ## the groups stored in the restore lists to
            ## restore the o_ and x_groups lists.
            else:
                print('invalid move - that returns to board to a previous state')
                for group in restore_o:
                    o_groups.append(group)
                for group in restore_x:
                    x_groups.append(group)
    if (player1_pass == 1) & (player2_pass == 1):
        gameover = 1
 
## Called to start a game
def main():
    ## Either 'o' or 'x', determines who's turn it is
    global xoro
    ## The opposite of xoro, determines who's turn it is not
    global notxoro
    ## Game State Current, the current layout of the board
    ## This value is two-dimensional list, the higher dimension being
    ## lists representing the rows and the lower dimension being
    ## strings representing individual positions on the board.
    ## These strings are either '-', 'o', or 'x'
    global gsc
    ## 0 or 1, determins whether the current game is ongoing or ended
    global gameover
    ## Game State Future, same setup as gsc, used for testing the
    ## waters of a new move, to see if that move is valid, before
    ## gsc is edited to reflect that move
    global gsf
    ## Two-dimensional lists, the higher dimension being groups, the
    ## lower dimension being lists of board positions in a particular
    ## group
    global o_groups
    global x_groups
    ## Groups of empty positions
    global non_groups
    ## String containing all the game states encountered in a particular
    ## game, used to check validity of moves
    global gscache
    ## 0 or 1, for whether the player has passed their turn or not
    global player1_pass
    global player2_pass
    ## Integer value reflecting the score of a player
    global o_points
    global x_points
 
    ## Creates a blank game state - a blank board
    gsc = initalize()
    gsf = initalize()
    ## Sets initial values
    o_groups = []
    x_groups = []
    non_groups = []
    gscache = ''
    player1_pass = 0
    player2_pass = 0
    gameover = 0
    o_points = 0
    x_points = 0
 
    ## Gives players turns until the end of the game
    ## (that is, until both players pass, one after
    ## the other)
    while gameover != 1:
 
        ## Set it as o-player's turn
        xoro = 'o'
        notxoro = 'x'
        print()
        printboard(gsc)
 
        turn()
        if gameover == 1:
            break
 
        ## Sets it as x-player's turn
        xoro = 'x'
        notxoro = 'o'
        print()
        printboard(gsc)
 
        turn()
 
    ## Counts the score of both players
    count()
    print()
    print('final board:')
    print()
    printboard(gsc)
    print()
    print('o points: ',str(o_points))
    print('x points: ',str(x_points))
    ## Determines the winner
    if o_points > x_points:
        print('o wins')
    elif x_points > o_points:
        print('x wins')
    else:
        print('tie')
 
## Finally something that is not a function!
## This while loop will start new games for as
## long as the user choses to.
while gameon == 1:
    main()
    hold = 1
    while hold == 1:
        yn = raw_input('play again (y/n)? ')
        if yn == 'n':
            gameon = 0
            hold = 0
        elif yn == 'y':
            hold = 0
        else:
            print('invalid')


()
- - - - - 
- - - - - 
- - - - - 
- - - - - 
- - - - - 
()
place for o
Place or pass (l/a)? l
x: 1
y: 2
()
- - - - - 
- - - - - 
- o - - - 
- - - - - 
- - - - - 
()
place for x
Place or pass (l/a)? l
x: 3
y: 4
()
- - - - - 
- - - - - 
- o - - - 
- - - - - 
- - - x - 
()
place for o
Place or pass (l/a)? l
x: 4
y: 4
()
- - - - - 
- - - - - 
- o - - - 
- - - - - 
- - - x o 
()
place for x
Place or pass (l/a)? a
()
- - - - - 
- - - - - 
- o - - - 
- - - - - 
- - - x o 
()
place for o
Place or pass (l/a)? a
()
final board:
()
- - - - - 
- - - - - 
- o - - - 
- - - - - 
- - - x o 
()
('o points: ', '2')
('x points: ', '1')
o wins
play again (y/n)? n


In [6]:
#https://github.com/Rochester-NRT/RocAlphaGo/wiki

In [7]:
#Policy network
from keras.models import Sequential, Model
from keras.layers import convolutional, merge, Input, BatchNormalization
from keras.layers.core import Activation, Flatten
from AlphaGo.util import flatten_idx
from AlphaGo.models.nn_util import Bias, NeuralNetBase, neuralnet
import numpy as np


@neuralnet
class CNNPolicy(NeuralNetBase):
	"""uses a convolutional neural network to evaluate the state of the game
	and compute a probability distribution over the next action
	"""

	def _select_moves_and_normalize(self, nn_output, moves, size):
		"""helper function to normalize a distribution over the given list of moves
		and return a list of (move, prob) tuples
		"""
		if len(moves) == 0:
			return []
		move_indices = [flatten_idx(m, size) for m in moves]
		# get network activations at legal move locations
		distribution = nn_output[move_indices]
		distribution = distribution / distribution.sum()
		return zip(moves, distribution)

	def batch_eval_state(self, states, moves_lists=None):
		"""Given a list of states, evaluates them all at once to make best use of GPU
		batching capabilities.

		Analogous to [eval_state(s) for s in states]

		Returns: a parallel list of move distributions as in eval_state
		"""
		n_states = len(states)
		if n_states == 0:
			return []
		state_size = states[0].size
		if not all([st.size == state_size for st in states]):
			raise ValueError("all states must have the same size")
		# concatenate together all one-hot encoded states along the 'batch' dimension
		nn_input = np.concatenate([self.preprocessor.state_to_tensor(s) for s in states], axis=0)
		# pass all input through the network at once (backend makes use of batches if len(states) is large)
		network_output = self.forward(nn_input)
		# default move lists to all legal moves
		moves_lists = moves_lists or [st.get_legal_moves() for st in states]
		results = [None] * n_states
		for i in range(n_states):
			results[i] = self._select_moves_and_normalize(network_output[i], moves_lists[i], state_size)
		return results

	def eval_state(self, state, moves=None):
		"""Given a GameState object, returns a list of (action, probability) pairs
		according to the network outputs

		If a list of moves is specified, only those moves are kept in the distribution
		"""
		tensor = self.preprocessor.state_to_tensor(state)
		# run the tensor through the network
		network_output = self.forward(tensor)
		moves = moves or state.get_legal_moves()
		return self._select_moves_and_normalize(network_output[0], moves, state.size)

	@staticmethod
	def create_network(**kwargs):
		"""construct a convolutional neural network.

		Keword Arguments:
		- input_dim:         	depth of features to be processed by first layer (no default)
		- board:             	width of the go board to be processed (default 19)
		- filters_per_layer: 	number of filters used on every layer (default 128)
		- layers:            	number of convolutional steps (default 12)
		- filter_width_K:    	(where K is between 1 and <layers>) width of filter on
								layer K (default 3 except 1st layer which defaults to 5).
								Must be odd.
		"""
		defaults = {
			"board": 19,
			"filters_per_layer": 128,
			"layers": 12,
			"filter_width_1": 5
		}
		# copy defaults, but override with anything in kwargs
		params = defaults
		params.update(kwargs)

		# create the network:
		# a series of zero-paddings followed by convolutions
		# such that the output dimensions are also board x board
		network = Sequential()

		# create first layer
		network.add(convolutional.Convolution2D(
			input_shape=(params["input_dim"], params["board"], params["board"]),
			nb_filter=params["filters_per_layer"],
			nb_row=params["filter_width_1"],
			nb_col=params["filter_width_1"],
			init='uniform',
			activation='relu',
			border_mode='same'))

		# create all other layers
		for i in range(2, params["layers"] + 1):
			# use filter_width_K if it is there, otherwise use 3
			filter_key = "filter_width_%d" % i
			filter_width = params.get(filter_key, 3)
			network.add(convolutional.Convolution2D(
				nb_filter=params["filters_per_layer"],
				nb_row=filter_width,
				nb_col=filter_width,
				init='uniform',
				activation='relu',
				border_mode='same'))

		# the last layer maps each <filters_per_layer> feature to a number
		network.add(convolutional.Convolution2D(
			nb_filter=1,
			nb_row=1,
			nb_col=1,
			init='uniform',
			border_mode='same'))
		# reshape output to be board x board
		network.add(Flatten())
		# add a bias to each board location
		network.add(Bias())
		# softmax makes it into a probability distribution
		network.add(Activation('softmax'))

		return network


@neuralnet
class ResnetPolicy(CNNPolicy):
	"""Residual network architecture as per He at al. 2015
	"""
	@staticmethod
	def create_network(**kwargs):
		"""construct a convolutional neural network with Resnet-style skip connections.
		Arguments are the same as with the default CNNPolicy network, except the default
		number of layers is 20 plus a new n_skip parameter

		Keword Arguments:
		- input_dim:         	depth of features to be processed by first layer (no default)
		- board:             	width of the go board to be processed (default 19)
		- filters_per_layer: 	number of filters used on every layer (default 128)
		- layers:            	number of convolutional steps (default 20)
		- filter_width_K:    	(where K is between 1 and <layers>) width of filter on
								layer K (default 3 except 1st layer which defaults to 5).
								Must be odd.
		- n_skip_K:             (where K is as in filter_width_K) number of convolutional
								layers to skip with the linear path starting at K. Only valid
								at K >= 1. (Each layer defaults to 1)

		Note that n_skip_1=s means that the next valid value of n_skip_* is 3

		A diagram may help explain (numbers indicate layer):

			1             2              3                   4              5              6
		I--C -- B -- R -- C -- B -- R -- C -- M -- B -- R -- C -- B -- R -- C -- B -- R -- C -- M  ...  M  -- R -- F -- O
			\___________________________/ \____________________________________________________/ \ ... /
					[n_skip_1 = 2]                             [n_skip_3 = 3]

		I - input
		B - BatchNormalization
		R - ReLU
		C - Conv2D
		F - Flatten
		O - output
		M - merge

		The input is always passed through a Conv2D layer, the output of which layer is counted as '1'.
		Each subsequent [R -- C] block is counted as one 'layer'. The 'merge' layer isn't counted; hence
		if n_skip_1 is 2, the next valid skip parameter is n_skip_3, which will start at the output
		of the merge
		"""
		defaults = {
			"board": 19,
			"filters_per_layer": 128,
			"layers": 20,
			"filter_width_1": 5
		}
		# copy defaults, but override with anything in kwargs
		params = defaults
		params.update(kwargs)

		# create the network using Keras' functional API,
		# since this isn't 'Sequential'
		model_input = Input(shape=(params["input_dim"], params["board"], params["board"]))

		# create first layer
		convolution_path = convolutional.Convolution2D(
			input_shape=(),
			nb_filter=params["filters_per_layer"],
			nb_row=params["filter_width_1"],
			nb_col=params["filter_width_1"],
			init='uniform',
			activation='linear',  # relu activations done inside resnet modules
			border_mode='same')(model_input)

		def add_resnet_unit(path, K, **params):
			"""Add a resnet unit to path starting at layer 'K',
			adding as many (ReLU + Conv2D) modules as specified by n_skip_K

			Returns new path and next layer index, i.e. K + n_skip_K, in a tuple
			"""
			# loosely based on https://github.com/keunwoochoi/residual_block_keras
			# (see also keras docs here: http://keras.io/getting-started/functional-api-guide/#all-models-are-callable-just-like-layers)

			block_input = path
			# use n_skip_K if it is there, default to 1
			skip_key = "n_skip_%d" % K
			n_skip = params.get(skip_key, 1)
			for i in range(n_skip):
				layer = K + i
				# add BatchNorm
				path = BatchNormalization()(path)
				# add ReLU
				path = Activation('relu')(path)
				# use filter_width_K if it is there, otherwise use 3
				filter_key = "filter_width_%d" % layer
				filter_width = params.get(filter_key, 3)
				# add Conv2D
				path = convolutional.Convolution2D(
					nb_filter=params["filters_per_layer"],
					nb_row=filter_width,
					nb_col=filter_width,
					init='uniform',
					activation='linear',
					border_mode='same')(path)
			# Merge 'input layer' with the path
			path = merge([block_input, path], mode='sum')
			return path, K + n_skip

		# create all other layers
		layer = 1
		while layer < params['layers']:
			convolution_path, layer = add_resnet_unit(convolution_path, layer, **params)
		if layer > params['layers']:
			print "Due to skipping, ended with {} layers instead of {}".format(layer, params['layers'])

		# since each layer's activation was linear, need one more ReLu
		convolution_path = Activation('relu')(convolution_path)

		# the last layer maps each <filters_per_layer> featuer to a number
		convolution_path = convolutional.Convolution2D(
			nb_filter=1,
			nb_row=1,
			nb_col=1,
			init='uniform',
			border_mode='same')(convolution_path)
		# flatten output
		network_output = Flatten()(convolution_path)
		# add a bias to each board location
		network_output = Bias()(network_output)
		# softmax makes it into a probability distribution
		network_output = Activation('softmax')(network_output)

		return Model(input=[model_input], output=[network_output])


SyntaxError: invalid syntax (<ipython-input-7-b96253035da6>, line 240)

In [8]:
#Value network
from keras.models import Sequential
from keras.layers import convolutional
from keras.layers.core import Dense, Flatten
# from SGD_exponential_decay import SGD_exponential_decay as SGD

### Parameters obtained from paper ###
K = 152                        # depth of convolutional layers
LEARNING_RATE = .003           # initial learning rate
DECAY = 8.664339379294006e-08  # rate of exponential learning_rate decay


class value_trainer:
    def __init__(self):
        self.model = Sequential()
        self.model.add(convolutional.Convolution2D(
            input_shape=(49, 19, 19), nb_filter=K, nb_row=5, nb_col=5,
            init='uniform', activation='relu', border_mode='same'))
        for i in range(2, 13):
            self.model.add(convolutional.Convolution2D(
                nb_filter=K, nb_row=3, nb_col=3,
                init='uniform', activation='relu', border_mode='same'))

        self.model.add(convolutional.Convolution2D(
            nb_filter=1, nb_row=1, nb_col=1,
            init='uniform', activation='linear', border_mode='same'))
        self.model.add(Flatten())
        self.model.add(Dense(256, init='uniform'))
        self.model.add(Dense(1, init='uniform', activation="tanh"))

        # sgd = SGD(lr=LEARNING_RATE, decay=DECAY)
        # self.model.compile(loss='mean_squared_error', optimizer=sgd)

    def get_samples(self):
        # TODO non-terminating loop that draws training samples uniformly at random
        pass

    def train(self):
        # TODO use self.model.fit_generator to train from data source
        pass

if __name__ == '__main__':
    trainer = value_trainer()
    # TODO command line instantiation


Using Theano backend.


In [9]:
##NN_util
from keras import backend as K
from keras.models import model_from_json
from keras.engine.topology import Layer
from AlphaGo.preprocessing.preprocessing import Preprocess
import json


class NeuralNetBase(object):
	"""Base class for neural network classes handling feature processing, construction
	of a 'forward' function, etc.
	"""

	# keep track of subclasses to make generic saving/loading cleaner.
	# subclasses can be 'registered' with the @neuralnet decorator
	subclasses = {}

	def __init__(self, feature_list, **kwargs):
		"""create a neural net object that preprocesses according to feature_list and uses
		a neural network specified by keyword arguments (using subclass' create_network())

		optional argument: init_network (boolean). If set to False, skips initializing
		self.model and self.forward and the calling function should set them.
		"""
		self.preprocessor = Preprocess(feature_list)
		kwargs["input_dim"] = self.preprocessor.output_dim

		if kwargs.get('init_network', True):
			# self.__class__ refers to the subclass so that subclasses only
			# need to override create_network()
			self.model = self.__class__.create_network(**kwargs)
			# self.forward is a lambda function wrapping a Keras function
			self.forward = self._model_forward()

	def _model_forward(self):
		"""Construct a function using the current keras backend that, when given a batch
		of inputs, simply processes them forward and returns the output

		This is as opposed to model.compile(), which takes a loss function
		and training method.

		c.f. https://github.com/fchollet/keras/issues/1426
		"""
		# The uses_learning_phase property is True if the model contains layers that behave
		# differently during training and testing, e.g. Dropout or BatchNormalization.
		# In these cases, K.learning_phase() is a reference to a backend variable that should
		# be set to 0 when using the network in prediction mode and is automatically set to 1
		# during training.
		if self.model.uses_learning_phase:
			forward_function = K.function([self.model.input, K.learning_phase()], [self.model.output])

			# the forward_function returns a list of tensors
			# the first [0] gets the front tensor.
			return lambda inpt: forward_function([inpt, 0])[0]
		else:
			# identical but without a second input argument for the learning phase
			forward_function = K.function([self.model.input], [self.model.output])
			return lambda inpt: forward_function([inpt])[0]

	@staticmethod
	def load_model(json_file):
		"""create a new neural net object from the architecture specified in json_file
		"""
		with open(json_file, 'r') as f:
			object_specs = json.load(f)

		# Create object; may be a subclass of networks saved in specs['class']
		class_name = object_specs.get('class', 'CNNPolicy')
		try:
			network_class = NeuralNetBase.subclasses[class_name]
		except KeyError:
			raise ValueError("Unknown neural network type in json file: {}\n(was it registered with the @neuralnet decorator?)".format(network_class))

		# create new object
		new_net = network_class(object_specs['feature_list'], init_network=False)

		new_net.model = model_from_json(object_specs['keras_model'], custom_objects={'Bias': Bias})
		if 'weights_file' in object_specs:
			new_net.model.load_weights(object_specs['weights_file'])
		new_net.forward = new_net._model_forward()
		return new_net

	def save_model(self, json_file, weights_file=None):
		"""write the network model and preprocessing features to the specified file

		If a weights_file (.hdf5 extension) is also specified, model weights are also
		saved to that file and will be reloaded automatically in a call to load_model
		"""
		# this looks odd because we are serializing a model with json as a string
		# then making that the value of an object which is then serialized as
		# json again.
		# It's not as crazy as it looks. A Network has 2 moving parts - the
		# feature preprocessing and the neural net, each of which gets a top-level
		# entry in the saved file. Keras just happens to serialize models with JSON
		# as well. Note how this format makes load_model fairly clean as well.
		object_specs = {
			'class': self.__class__.__name__,
			'keras_model': self.model.to_json(),
			'feature_list': self.preprocessor.feature_list
		}
		if weights_file is not None:
			self.model.save_weights(weights_file)
			object_specs['weights_file'] = weights_file
		# use the json module to write object_specs to file
		with open(json_file, 'w') as f:
			json.dump(object_specs, f)


def neuralnet(cls):
	"""Class decorator for registering subclasses of NeuralNetBase
	"""
	NeuralNetBase.subclasses[cls.__name__] = cls
	return cls


class Bias(Layer):
	"""Custom keras layer that simply adds a scalar bias to each location in the input

	Largely copied from the keras docs:
	http://keras.io/layers/writing-your-own-keras-layers/#writing-your-own-keras-layers
	"""
	def __init__(self, **kwargs):
		super(Bias, self).__init__(**kwargs)

	def build(self, input_shape):
		self.W = K.zeros(input_shape[1:])
		self.trainable_weights = [self.W]

	def call(self, x, mask=None):
		return x + self.W


ImportError: No module named AlphaGo.preprocessing.preprocessing

In [10]:
#mcts
from keras import backend as K
from keras.models import model_from_json
from keras.engine.topology import Layer
from AlphaGo.preprocessing.preprocessing import Preprocess
import json


class NeuralNetBase(object):
	"""Base class for neural network classes handling feature processing, construction
	of a 'forward' function, etc.
	"""

	# keep track of subclasses to make generic saving/loading cleaner.
	# subclasses can be 'registered' with the @neuralnet decorator
	subclasses = {}

	def __init__(self, feature_list, **kwargs):
		"""create a neural net object that preprocesses according to feature_list and uses
		a neural network specified by keyword arguments (using subclass' create_network())

		optional argument: init_network (boolean). If set to False, skips initializing
		self.model and self.forward and the calling function should set them.
		"""
		self.preprocessor = Preprocess(feature_list)
		kwargs["input_dim"] = self.preprocessor.output_dim

		if kwargs.get('init_network', True):
			# self.__class__ refers to the subclass so that subclasses only
			# need to override create_network()
			self.model = self.__class__.create_network(**kwargs)
			# self.forward is a lambda function wrapping a Keras function
			self.forward = self._model_forward()

	def _model_forward(self):
		"""Construct a function using the current keras backend that, when given a batch
		of inputs, simply processes them forward and returns the output

		This is as opposed to model.compile(), which takes a loss function
		and training method.

		c.f. https://github.com/fchollet/keras/issues/1426
		"""
		# The uses_learning_phase property is True if the model contains layers that behave
		# differently during training and testing, e.g. Dropout or BatchNormalization.
		# In these cases, K.learning_phase() is a reference to a backend variable that should
		# be set to 0 when using the network in prediction mode and is automatically set to 1
		# during training.
		if self.model.uses_learning_phase:
			forward_function = K.function([self.model.input, K.learning_phase()], [self.model.output])

			# the forward_function returns a list of tensors
			# the first [0] gets the front tensor.
			return lambda inpt: forward_function([inpt, 0])[0]
		else:
			# identical but without a second input argument for the learning phase
			forward_function = K.function([self.model.input], [self.model.output])
			return lambda inpt: forward_function([inpt])[0]

	@staticmethod
	def load_model(json_file):
		"""create a new neural net object from the architecture specified in json_file
		"""
		with open(json_file, 'r') as f:
			object_specs = json.load(f)

		# Create object; may be a subclass of networks saved in specs['class']
		class_name = object_specs.get('class', 'CNNPolicy')
		try:
			network_class = NeuralNetBase.subclasses[class_name]
		except KeyError:
			raise ValueError("Unknown neural network type in json file: {}\n(was it registered with the @neuralnet decorator?)".format(network_class))

		# create new object
		new_net = network_class(object_specs['feature_list'], init_network=False)

		new_net.model = model_from_json(object_specs['keras_model'], custom_objects={'Bias': Bias})
		if 'weights_file' in object_specs:
			new_net.model.load_weights(object_specs['weights_file'])
		new_net.forward = new_net._model_forward()
		return new_net

	def save_model(self, json_file, weights_file=None):
		"""write the network model and preprocessing features to the specified file

		If a weights_file (.hdf5 extension) is also specified, model weights are also
		saved to that file and will be reloaded automatically in a call to load_model
		"""
		# this looks odd because we are serializing a model with json as a string
		# then making that the value of an object which is then serialized as
		# json again.
		# It's not as crazy as it looks. A Network has 2 moving parts - the
		# feature preprocessing and the neural net, each of which gets a top-level
		# entry in the saved file. Keras just happens to serialize models with JSON
		# as well. Note how this format makes load_model fairly clean as well.
		object_specs = {
			'class': self.__class__.__name__,
			'keras_model': self.model.to_json(),
			'feature_list': self.preprocessor.feature_list
		}
		if weights_file is not None:
			self.model.save_weights(weights_file)
			object_specs['weights_file'] = weights_file
		# use the json module to write object_specs to file
		with open(json_file, 'w') as f:
			json.dump(object_specs, f)


def neuralnet(cls):
	"""Class decorator for registering subclasses of NeuralNetBase
	"""
	NeuralNetBase.subclasses[cls.__name__] = cls
	return cls


class Bias(Layer):
	"""Custom keras layer that simply adds a scalar bias to each location in the input

	Largely copied from the keras docs:
	http://keras.io/layers/writing-your-own-keras-layers/#writing-your-own-keras-layers
	"""
	def __init__(self, **kwargs):
		super(Bias, self).__init__(**kwargs)

	def build(self, input_shape):
		self.W = K.zeros(input_shape[1:])
		self.trainable_weights = [self.W]

	def call(self, x, mask=None):
		return x + self.W


ImportError: No module named AlphaGo.preprocessing.preprocessing

In [11]:
import os
import argparse
import json
import numpy as np
from shutil import copyfile
from keras.optimizers import SGD
from AlphaGo.ai import ProbabilisticPolicyPlayer
import AlphaGo.go as go
from AlphaGo.go import GameState
from AlphaGo.models.policy import CNNPolicy
from AlphaGo.preprocessing.preprocessing import Preprocess
from AlphaGo.util import flatten_idx


def make_training_pairs(player, opp, features, mini_batch_size, board_size=19):
	"""Make training pairs for batch of matches, utilizing player.get_moves (parallel form of
	player.get_move), which calls `CNNPolicy.batch_eval_state`.

	Args:
	player -- player that we're always updating
	opp -- batch opponent
	feature_list -- game features to be one-hot encoded
	mini_batch_size -- number of games in mini-batch

	Return:
	X_list -- list of 1-hot board states associated with moves.
	y_list -- list of 1-hot moves associated with board states.
	winners -- list of winners associated with each game in batch
	"""

	def do_move(states, states_prev, moves, X_list, y_list, player_color):
		bsize_flat = bsize * bsize
		for st, st_prev, mv, X, y in zip(states, states_prev, moves, X_list, y_list):
			if not st.is_end_of_game:
				# Only do more moves if not end of game already
				st.do_move(mv)
				if st.current_player != player_color and mv is not go.PASS_MOVE:
					# Convert move to one-hot
					state_1hot = preprocessor.state_to_tensor(st_prev)
					move_1hot = np.zeros(bsize_flat)
					move_1hot[flatten_idx(mv, bsize)] = 1
					X.append(state_1hot)
					y.append(move_1hot)
		return states, X_list, y_list

	# Lists of game training pairs (1-hot)
	X_list = [list() for _ in xrange(mini_batch_size)]
	y_list = [list() for _ in xrange(mini_batch_size)]
	preprocessor = Preprocess(features)
	bsize = player.policy.model.input_shape[-1]
	states = [GameState(size=board_size) for i in xrange(mini_batch_size)]
	# Randomly choose who goes first (i.e. color of 'player')
	player_color = np.random.choice([go.BLACK, go.WHITE])
	player1, player2 = (player, opp) if player_color == go.BLACK else \
		(opp, player)
	while True:
		# Cache states before moves
		states_prev = [st.copy() for st in states]
		# Get moves (batch)
		moves_black = player1.get_moves(states)
		# Do moves (black)
		states, X_list, y_list = do_move(states, states_prev, moves_black, X_list, y_list, player_color)
		# Do moves (white)
		moves_white = player2.get_moves(states)
		states, X_list, y_list = do_move(states, states_prev, moves_white, X_list, y_list, player_color)
		# If all games have ended, we're done. Get winners.
		done = [st.is_end_of_game for st in states]
		if all(done):
			break
	winners = [st.get_winner() for st in states]
	# Concatenate tensors across turns within each game
	for i in xrange(mini_batch_size):
		X_list[i] = np.concatenate(X_list[i], axis=0)
		y_list[i] = np.vstack(y_list[i])
	return X_list, y_list, winners


def train_batch(player, X_list, y_list, winners, lr):
	"""Given the outcomes of a mini-batch of play against a fixed opponent,
		update the weights with reinforcement learning.

		Args:
		player -- player object with policy weights to be updated
		X_list -- List of one-hot encoded states.
		y_list -- List of one-hot encoded actions (to pair with X_list).
		winners -- List of winners corresponding to each item in
					training_pairs_list
		lr -- Keras learning rate

		Return:
		player -- same player, with updated weights.
	"""

	for X, y, winner in zip(X_list, y_list, winners):
		# Update weights in + direction if player won, and - direction if player lost.
		# Setting learning rate negative is hack for negative weights update.
		if winner == -1:
			player.policy.model.optimizer.lr.set_value(-lr)
		else:
			player.policy.model.optimizer.lr.set_value(lr)
		player.policy.model.fit(X, y, nb_epoch=1, batch_size=len(X))


def run_training(cmd_line_args=None):
	parser = argparse.ArgumentParser(description='Perform reinforcement learning to improve given policy network. Second phase of pipeline.')
	parser.add_argument("model_json", help="Path to policy model JSON.")
	parser.add_argument("initial_weights", help="Path to HDF5 file with inital weights (i.e. result of supervised training).")
	parser.add_argument("out_directory", help="Path to folder where the model params and metadata will be saved after each epoch.")
	parser.add_argument("--learning-rate", help="Keras learning rate (Default: .03)", type=float, default=.03)
	parser.add_argument("--policy-temp", help="Distribution temperature of players using policies (Default: 0.67)", type=float, default=0.67)
	parser.add_argument("--save-every", help="Save policy as a new opponent every n batches (Default: 500)", type=int, default=500)
	parser.add_argument("--game-batch", help="Number of games per mini-batch (Default: 20)", type=int, default=20)
	parser.add_argument("--iterations", help="Number of training batches/iterations (Default: 10000)", type=int, default=10000)
	parser.add_argument("--resume", help="Load latest weights in out_directory and resume", default=False, action="store_true")
	parser.add_argument("--verbose", "-v", help="Turn on verbose mode", default=False, action="store_true")
	# Baseline function (TODO) default lambda state: 0  (receives either file
	# paths to JSON and weights or None, in which case it uses default baseline 0)
	if cmd_line_args is None:
		args = parser.parse_args()
	else:
		args = parser.parse_args(cmd_line_args)

	ZEROTH_FILE = "weights.00000.hdf5"

	if args.resume:
		if not os.path.exists(os.path.join(args.out_directory, "metadata.json")):
			raise ValueError("Cannot resume without existing output directory")

	if not os.path.exists(args.out_directory):
		if args.verbose:
			print "creating output directory {}".format(args.out_directory)
		os.makedirs(args.out_directory)

	if not args.resume:
		# make a copy of weights file, "weights.00000.hdf5" in the output directory
		copyfile(args.initial_weights, os.path.join(args.out_directory, ZEROTH_FILE))
		if args.verbose:
			print "copied {} to {}".format(args.initial_weights, os.path.join(args.out_directory, ZEROTH_FILE))
		player_weights = ZEROTH_FILE
	else:
		# if resuming, we expect initial_weights to be just a "weights.#####.hdf5" file, not a full path
		args.initial_weights = os.path.join(args.out_directory, os.path.basename(args.initial_weights))
		if not os.path.exists(args.initial_weights):
			raise ValueError("Cannot resume; weights {} do not exist".format(args.initial_weights))
		elif args.verbose:
			print "Resuming with weights {}".format(args.initial_weights)
		player_weights = os.path.basename(args.initial_weights)

	# Set initial conditions
	policy = CNNPolicy.load_model(args.model_json)
	policy.model.load_weights(args.initial_weights)
	player = ProbabilisticPolicyPlayer(policy, temperature=args.policy_temp)
	features = policy.preprocessor.feature_list

	# different opponents come from simply changing the weights of
	# opponent.policy.model "behind the scenes"
	opp_policy = CNNPolicy.load_model(args.model_json)
	opponent = ProbabilisticPolicyPlayer(opp_policy, temperature=args.policy_temp)

	if args.verbose:
		print "created player and opponent with temperature {}".format(args.policy_temp)

	if not args.resume:
		metadata = {
			"model_file": args.model_json,
			"init_weights": args.initial_weights,
			"learning_rate": args.learning_rate,
			"temperature": args.policy_temp,
			"game_batch": args.game_batch,
			"opponents": [ZEROTH_FILE],  # which weights from which to sample an opponent each batch
			"win_ratio": {}  # map from player to tuple of (opponent, win ratio) Useful for validating in lieu of 'accuracy/loss'
		}
	else:
		with open(os.path.join(args.out_directory, "metadata.json"), "r") as f:
			metadata = json.load(f)

	def save_metadata():
		with open(os.path.join(args.out_directory, "metadata.json"), "w") as f:
			json.dump(metadata, f)

	# Set SGD and compile
	sgd = SGD(lr=args.learning_rate)
	player.policy.model.compile(loss='binary_crossentropy', optimizer=sgd)
	board_size = player.policy.model.input_shape[-1]
	for i_iter in xrange(1, args.iterations + 1):
		# Train mini-batches by randomly choosing opponent from pool (possibly self)
		# and playing game_batch games against them
		opp_weights = np.random.choice(metadata["opponents"])
		opp_path = os.path.join(args.out_directory, opp_weights)
		# load new weights into opponent, but otherwise its the same
		opponent.policy.model.load_weights(opp_path)
		if args.verbose:
			print "Batch {}\tsampled opponent is {}".format(i_iter, opp_weights)
		# Make training pairs and do RL
		X_list, y_list, winners = make_training_pairs(player, opponent, features, args.game_batch, board_size)
		win_ratio = np.sum(np.array(winners) == 1) / float(args.game_batch)
		metadata["win_ratio"][player_weights] = (opp_weights, win_ratio)
		train_batch(player, X_list, y_list, winners, args.learning_rate)
		# Save intermediate models
		player_weights = "weights.%05d.hdf5" % i_iter
		player.policy.model.save_weights(os.path.join(args.out_directory, player_weights))
		# add player to batch of oppenents once in a while
		if i_iter % args.save_every == 0:
			metadata["opponents"].append(player_weights)
		save_metadata()

if __name__ == '__main__':
	run_training()


SyntaxError: invalid syntax (<ipython-input-11-074651e65ed2>, line 130)